In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [7]
commitnumber = de5d43e
molecule = {'o3': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3
  ng_adju = [0, 0]
  ng_refs = [2, 5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.6, 1.75), (1.55, 1.66, 1.7, 1.75, 1.8)]
  wgt = [(0.35, 0.6), (0.5, 0.55, 0.7, 0.9, 1.0)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.038974                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.030558
0.000750      2.0 -0.043361                  2.0 -0.034023
0.001052      3.0 -0.051819                  3.0 -0.040772
0.001476      4.0 -0.059933                  4.0 -0.047277
0.002070      5.0 -0.067716                  5.0 -0.053555
0.002904      6.0 -0.075181                  6.0 -0.059619
0.004074      7.0 -0.082304                  7.0 -0.065440
0.005714      8.0 -0.089080                  8.0 -0.071026
0.008015      9.0 -0.095516                  9.0 -0.076353
0.011243     10.0 -0.101634                 10.0 -0.081446
0.015771     11.0 -0.107581                 11.0 -0.086281
0.022122     12.0 -0.113372                 12.0 -0.090896
0.031031     13.0 -0.119173                 13.0 -0.095300
0.043528     14.0 -0.124927                 14.0 -0.099377
0.061057     15.0 -0.126906                 15.0 -0.100703
0.085645     16.0 -0.117832                 16.0 -0.094512
0.120136     17.0 -0.069908                 17.0 -0.057833
0.168516     18.0  0.032348                 18.0  0.025780
0.236378     19.0  0.199115                 19.0  0.169012
0.331549     20.0  0.442457                 20.0  0.387200
0.465100     21.0  0.775433                 21.0  0.696530
0.652400     22.0  1.217520                 22.0  1.113044
0.915100     23.0  1.732497                 23.0  1.608654
1.283650     24.0  1.993833                 24.0  1.911636
1.800600     25.0  1.790441                 25.0  1.780755
2.525700     26.0  1.436163                 26.0  1.432863
3.542800     27.0  1.061659                 27.0  1.053281
4.969550     28.0  0.744368                 28.0  0.752651
6.970850     29.0  0.522055                 29.0  0.553431
9.778100     30.0  0.351804                 30.0  0.397058
13.715850    31.0  0.178967                 31.0  0.220419
19.239350    32.0  0.010305                 32.0  0.034989
26.987250    33.0 -0.108022                 33.0 -0.100250
37.855300    34.0 -0.191344                 34.0 -0.199663
53.100050    35.0 -0.261042                 35.0 -0.287349
73.887500    36.0 -0.282583                 36.0 -0.323316
97.662500    37.0 -0.250037                 37.0 -0.290682
121.437500   38.0 -0.216483                 38.0 -0.249834
145.212500   39.0 -0.201254                 39.0 -0.226266
168.987500   40.0 -0.180020                 40.0 -0.195907
192.762500   41.0 -0.144146                 41.0 -0.153087
216.537500   42.0 -0.114265                 42.0 -0.119580
240.312500   43.0 -0.093807                 43.0 -0.096883
264.087500   44.0 -0.074862                 44.0 -0.076315
287.862500   45.0 -0.059860                 45.0 -0.060303
311.637500   46.0 -0.049904                 46.0 -0.049701
335.412500   47.0 -0.041454                 47.0 -0.040794
359.187500   48.0 -0.033650                 48.0 -0.032687
382.962500   49.0 -0.026999                 49.0 -0.025847
406.737500   50.0 -0.021379                 50.0 -0.020113
430.512500   51.0 -0.016308                 51.0 -0.014988
454.287500   52.0 -0.011774                 52.0 -0.010450
478.062500   53.0 -0.007761                 53.0 -0.006470
501.837500   54.0 -0.004288                 54.0 -0.003050
525.612500   55.0 -0.001214                 55.0 -0.000039
549.387500   56.0  0.001574                 56.0  0.002672
573.162500   57.0  0.004125                 57.0  0.005137
596.937500   58.0  0.006484                 58.0  0.007402
620.712500   59.0  0.008655                 59.0  0.009468
644.487500   60.0  0.010675                 60.0  0.011378
668.262500   61.0  0.012563                 61.0  0.013150
692.037500   62.0  0.014295                 62.0  0.014762
715.812500   63.0  0.015912                 63.0  0.016255
739.587500   64.0  0.017443                 64.0  0.017662
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -23.045620  0.000000 -23.045620        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -22.641528  0.000002   
6.244000e-04 2     -23.045625  0.000001 -23.045623 -22.641532  0.000003   
8.759000e-04 3     -23.045627  0.000002 -23.045625 -22.641533  0.000003   
1.228600e-03 4     -23.045630  0.000003 -23.045627 -22.641535  0.000004   
1.723400e-03 5     -23.045635  0.000005 -23.045630 -22.641540  0.000005   
2.417400e-03 6     -23.045644  0.000008 -23.045636 -22.641546  0.000008   
3.390900e-03 7     -23.045658  0.000013 -23.045644 -22.641557  0.000011   
4.756500e-03 8     -23.045679  0.000021 -23.045658 -22.641573  0.000017   
6.672000e-03 9     -23.045712  0.000034 -23.045678 -22.641598  0.000026   
9.358900e-03 10    -23.045763  0.000054 -23.045708 -22.641637  0.000040   
1.312780e-02 11    -23.045839  0.000086 -23.045754 -22.641695  0.000063   
1.841450e-02 12    -23.045955  0.000134 -23.045821 -22.641784  0.000097   
2.583020e-02 13    -23.046128  0.000207 -23.045921 -22.641916  0.000149   
3.623230e-02 14    -23.046387  0.000320 -23.046068 -22.642114  0.000230   
5.082340e-02 15    -23.046773  0.000490 -23.046284 -22.642408  0.000352   
7.129060e-02 16    -23.047354  0.000763 -23.046591 -22.642852  0.000552   
1.000000e-01 17    -23.048300  0.001308 -23.046992 -22.643574  0.000953   
1.402710e-01 18    -23.049828  0.002503 -23.047325 -22.644747  0.001849   
1.967600e-01 19    -23.052095  0.004986 -23.047109 -22.646481  0.003756   
2.759970e-01 20    -23.055152  0.009912 -23.045240 -22.648785  0.007647   
3.871000e-01 21    -23.058719  0.019302 -23.039417 -22.651350  0.015308   
5.431000e-01 22    -23.061658  0.036571 -23.025087 -22.653084  0.029915   
7.617000e-01 23    -23.060733  0.067172 -22.993560 -22.651110  0.056766   
1.068500e+00 24    -23.049717  0.119120 -22.930597 -22.640092  0.104217   
1.498800e+00 25    -23.024793  0.195826 -22.828967 -22.616534  0.178109   
2.102400e+00 26    -22.989374  0.288425 -22.700949 -22.582876  0.271790   
2.949000e+00 27    -22.946132  0.389210 -22.556922 -22.541273  0.373897   
4.136600e+00 28    -22.900467  0.492899 -22.407568 -22.496783  0.477597   
5.802500e+00 29    -22.860691  0.600015 -22.260676 -22.457548  0.586904   
8.139200e+00 30    -22.834582  0.718410 -22.116172 -22.431031  0.713592   
1.141700e+01 31    -22.833744  0.854169 -21.979574 -22.428542  0.865288   
1.601470e+01 32    -22.881561  0.999458 -21.882103 -22.473557  1.030363   
2.246400e+01 33    -23.012168  1.137938 -21.874231 -22.601233  1.184771   
3.151050e+01 34    -23.261521  1.271532 -21.989989 -22.851699  1.327797   
4.420010e+01 35    -23.685476  1.407865 -22.277612 -23.290887  1.466824   
6.200000e+01 36    -24.356033  1.528008 -22.828025 -24.016853  1.586845   
8.577500e+01 37    -25.235308  1.611440 -23.623868 -25.012077  1.671412   
1.095500e+02 38    -25.990446  1.662393 -24.328053 -25.883256  1.723855   
1.333250e+02 39    -26.640613  1.702874 -24.937739 -26.628404  1.765317   
1.571000e+02 40    -27.241178  1.736644 -25.504534 -27.300596  1.800208   
1.808750e+02 41    -27.779946  1.768419 -26.011527 -27.884771  1.832589   
2.046500e+02 42    -28.224245  1.806756 -26.417489 -28.353120  1.869751   
2.284250e+02 43    -28.595373  1.856076 -26.739296 -28.736277  1.916098   
2.522000e+02 44    -28.917807  1.914320 -27.003487 -29.063508  1.970447   
2.759750e+02 45    -29.190797  1.976474 -27.214323 -29.336422  2.028412   
2.997500e+02 46    -29.423699  2.040791 -27.382908 -29.566292  2.088431   
3.235250e+02 47    -29.632425  2.108972 -27.523452 -29.769982  2.152132   
3.473000e+02 48    -29.820538  2.180336 -27.640201 -29.951668  2.218919   
3.710750e+02 49    -29.988327  2.253357 -27.734970 -30.112187  2.287370   
3.948500e+02 50    -30.138870  2.327864 -27.811006 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')